# Problem 1

In [ ]:
import numpy as np
import copy
class SpellChecker:
    def __init__(self,beam,longest_word_lenth=45,tolerate_num_of_error=3):
        self.beam = beam
        self.lextree=None
        self.dist_fun=lambda *args: int(args[0] != args[1])
        self.longest_word_lenth=longest_word_lenth
        self.tolerate_num_of_error=tolerate_num_of_error

    def fit(self,lextree):
        self.lextree=lextree
        self.nodes = []
        self.get_nodes(self.lextree)
        self.transitions = {}
        self.get_={}
        to_children={}
        n_nodes = len(self.nodes)
        self.word_ends = []
        for i in range(n_nodes):
            n = self.nodes[i]
            if n.property == 2:
                self.word_ends.append(i)
            self.get_children[i]=[]
            if len(n.next) > 0:
                for child in n.next:
                    self.get_children[i].append(self.nodes.index(child))
                    self.transitions[self.nodes.index(child)] = i
        self.transitions[0]=0
    def correct_single_word(self,word):
        word='*'+word
        costs=np.full((len(self.nodes),len(word)),np.inf)
        for i in range(len(word)):
            costs[0][i]=i
        for i in range(len(word)):
            for j in range(1,len(self.nodes)):
                dist=self.distance(self.nodes[j].val, word[i])
                cost=0
                if i==0:
                    if self.previous[j]==0:
                        costs[j][i]=dist
                    else:
                        costs[j][i]=costs[self.previous[j]][i]+dist
                elif i==1:
                    if dist==0:
                        cost=np.min([costs[j][i-1]+1,costs[self.previous[j]][i-1]])
                    else:
                        cost=np.min([costs[j][i-1],costs[self.previous[j]][i-1]])
                else:
                    if dist==0:
                        cost=np.min([costs[j][i-1]+1,costs[self.previous[j]][i-1],
                                 costs[self.previous[j]][i-2]+1])
                    else:
                        cost=np.min([costs[j][i-1],costs[self.previous[j]][i-1],
                                 costs[self.previous[j]][i-2]+1,costs[self.previous[j]][i]])
                costs[j][i]=cost+dist
    def correct_single_word(self,word):
        word='*'+word
        costs=np.full((len(self.nodes),len(word)),np.inf)
        for i in range(len(word)):
            costs[0][i]=i
        for i in range(len(word)):
            for j in range(1,len(self.nodes)):
                dist=self.distance(self.nodes[j].val, word[i])
                cost=0
                if i==0:
                    if self.previous[j]==0:
                        costs[j][i]=dist
                    else:
                        costs[j][i]=costs[self.previous[j]][i]+dist
                elif i==1:
                    if dist==0:
                        cost=np.min([costs[j][i-1]+1,costs[self.previous[j]][i-1]])
                    else:
                        cost=np.min([costs[j][i-1],costs[self.previous[j]][i-1]])
                else:
                    if dist==0:
                        cost=np.min([costs[j][i-1]+1,costs[self.previous[j]][i-1],
                                 costs[self.previous[j]][i-2]+1])
                    else:
                        cost=np.min([costs[j][i-1],costs[self.previous[j]][i-1],
                                 costs[self.previous[j]][i-2]+1,costs[self.previous[j]][i]])
                costs[j][i]=cost+dist
    def mute_nodes(self,r):
        self.muted_nodes[r]=1
        to_mute_children=self.get_children[r]
        if len(to_mute_children)!=0:
            for child_idx in to_mute_children:
                if child_idx!=0:
                    self.mute_nodes(child_idx)
                
    def get_nodes(self, lexnode):
        lexnode.idx=len(self.nodes)
        self.nodes.append(lexnode)
        if lexnode.property==2:
            return
        for child in lexnode.next:
            self.get_nodes(child)
    
    def text_vertibe(self,x,longest=True,get_string=True):
        x = '*' + x
        n_cols = len(x)
        n_rows = len(self.nodes)
        costs = np.full((n_rows, n_cols), np.inf)
        costs[0,:]=range(n_cols)
        
        n_nodes = len(self.nodes)
        self.muted_nodes=np.zeros([n_nodes])
        for c in range(n_cols):
            self.recursion(self.lextree, c, x, costs)
        costs_for_words=[]
        for end_idx in self.word_ends:
            costs_for_words.append(costs[end_idx,-1])
        min_cost=min(costs_for_words)
        min_idxs=np.where(costs_for_words==min_cost)[0]
        if get_string:
            result=self.nodes_to_word(min_idxs,longest=longest)
            return result
        else:
            return min_idxs, min_cost
    def recursion(self, lexnode,c,x,costs):
        r=lexnode.idx
        if not self.muted_nodes[r]:
            current_cost=self.dist_fun(lexnode.val,x[c])
            if c==0:
                if self.transitions[r]==0:
                    costs[r,c]=current_cost
                else:
                    costs[r,c]=costs[self.transitions[r],c]+current_cost
            elif c==1:
                if current_cost==0:
                    to_check=[costs[r,c-1]+1, 
                              costs[self.transitions[r],c-1],
                             ] 
                else:
                    to_check=[costs[r,c-1], 
                              costs[self.transitions[r],c-1],
                             ] 
                nnn=np.argmin(to_check)
                costs[r,c]=to_check[nnn]+current_cost
            else:
                if current_cost==0:
                    to_check=[costs[r,c-1]+1, 
                              costs[self.transitions[r],c-1],
                              costs[self.transitions[r],c-2]+1 
                             ]
                else:
                    to_check=[costs[r,c-1],
                              costs[self.transitions[r],c-1],
                              costs[self.transitions[r],c],
                              costs[self.transitions[r],c-2]+1 
                             ]
                nnn=np.argmin(to_check)
                costs[r,c]=to_check[nnn]+current_cost
            
            if costs[r,c]>=self.tolerate_num_of_error+len(x):
                    self.mute_nodes(r)
        if lexnode.property==2:
            return
        for child in lexnode.next:
            if costs[r,c]<=self.beam+1:
                self.recursion(child,c,x,costs)

In [ ]:
    def fit(self,lextree):
        self.lextree=lextree
        self.get_tree_info(self.lextree)
        # get self.transitions
        self.transitions = {}
        self.get_children={}
        to_children={}
        n_nodes = len(self.nodes)
        self.word_ends = []
        for i in range(n_nodes):
            n = self.nodes[i]
            if n.state == -1:
                self.word_ends.append(i)
            self.get_children[i]=[]
            if len(n.next) > 0:
                for next_ones in n.next:
                    self.get_children[i].append(self.nodes.index(next_ones))
                    self.transitions[self.nodes.index(next_ones)] = i
        self.transitions[0]=0
    def do_segment(self,word,is_longest=True,best_result_number=10,cost=0):
        self.best_result_number=best_result_number
        deletion_cost = 1.0
        self.diagonal_cost= 1.0 
        insertion_cost = 1.0
        match_cost = 0.0
        space_cost = 0.0
        loop_cost = loop_cost
        n_cols = len(word)
        n_rows = len(self.nodes)
        costs = np.full(n_rows, np.inf)
        initial_cost=np.zeros(n_rows)
        for r in range(n_rows):
            current_cost=1
            if self.transitions[r]==0:
                initial_cost[r]=current_cost
            else:
                #if it is the middle nodes, then the cost is based on it previous node
                initial_cost[r]=initial_cost[self.transitions[r]]+current_cost
        initial_cost[0]=0
        #print(initial_cost)
        #we allow one insertion and one deletion in each word
        initial_nodes=self.get_children[0]
        next_to_check_nodes=[]
        for node in initial_nodes:
            next_to_check_nodes.append(node)
            next_to_check_nodes+=self.get_children[node]
        # we allow the deletion. therefore, each time, we update the current node and its child node
        initial_nodes=next_to_check_nodes
        return